In [3]:
import os
from PIL import Image

def process_vertical_strip(image_path, output_dir, count, target_size=224, stride=112):
    """
    针对竖长条图片（宽<224, 高>>224）的专用处理脚本
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    filename = os.path.splitext(os.path.basename(image_path))[0]

    try:
        img = Image.open(image_path).convert('RGB')
        img_w, img_h = img.size

        # ---Padding---
        if img_w < target_size:
            pad_left = (target_size - img_w) // 2
            new_img = Image.new('RGB', (target_size, img_h), (0, 0, 0))
            new_img.paste(img, (pad_left, 0))
            img = new_img
        elif img_w > target_size:
            scale = target_size / img_w
            new_h = int(img_h * scale)
            img = img.resize((target_size, new_h))

        img_w, img_h = img.size

        # --- Vertical Sliding ---
        start_count = count # 记录一下这张图开始时的编号，方便打印
        for y in range(0, img_h, stride):
            if y + target_size > img_h:
                box = (0, img_h - target_size, target_size, img_h)
            else:
                box = (0, y, target_size, y + target_size)

            patch = img.crop(box)

            save_name = f"{count}.jpg"
            patch.save(os.path.join(output_dir, save_name))
            count += 1

            if y + target_size >= img_h:
                break

        # 这里算一下这张图切了多少份，打印出来比较直观
        print(f"处理完成: {filename} | 切出 {count - start_count} 张 -> 当前累计编号: {count - 1}")
        return count

    except Exception as e:
        print(f"[Error] 处理 {filename} 失败: {e}")
        return count

# pre2.ipynb 的 main 函数修改版
# 目的：分别遍历 train 和 val 文件夹，对里面的长图进行切片

if __name__ == '__main__':
    # 1. 设置输入和输出路径
    # 输入：刚才 split.ipynb 生成的包含原图的 train/val 文件夹
    split_root = r"D:\work\Steel plate\WuGang"
    
    # 输出：最终喂给模型的切片文件夹
    final_output_root = r"D:\work\Steel plate\W"

    # 2. 双重循环：先遍历 [train, val]，再遍历类别 [1, 2, 3, 4]
    phases = ['train', 'val']
    
    for phase in phases:
        print(f"\n{'='*20} 正在处理阶段: {phase} {'='*20}")
        
        for i in range(1, 5): # 类别 1, 2, 3, 4
            class_name = str(i)
            print(f"\n--- 正在处理类别: {class_name} ---")

            # 拼接源路径: temp_split_raw/train/1
            source_folder = os.path.join(split_root, phase, class_name)
            
            # 拼接目标路径: final_dataset_ready/train/1
            # 注意：这里不需要加 "modified" 后缀了，保持 pytorch 标准结构最好
            target_folder = os.path.join(final_output_root, phase, class_name)

            # 初始化计数器 (每个文件夹重新计数)
            current_count = 1

            if not os.path.exists(source_folder):
                print(f"[跳过] 找不到源文件夹: {source_folder}")
                continue
            
            # 获取文件列表并排序
            file_list = os.listdir(source_folder)
            file_list.sort()

            for file_name in file_list:
                if file_name.lower().endswith(('.bmp', '.jpg', '.png', '.tif', '.tiff')):
                    full_path = os.path.join(source_folder, file_name)
                    
                    # 调用原本的切片函数
                    current_count = process_vertical_strip(full_path, target_folder, current_count)


==================== 正在处理阶段: train ====================

--- 正在处理类别: 1 ---
处理完成: 1-10 | 切出 147 张 -> 当前累计编号: 147
处理完成: 1-11 | 切出 144 张 -> 当前累计编号: 291
处理完成: 1-13 | 切出 148 张 -> 当前累计编号: 439
处理完成: 1-14 | 切出 145 张 -> 当前累计编号: 584
处理完成: 1-15 | 切出 139 张 -> 当前累计编号: 723
处理完成: 1-18 | 切出 144 张 -> 当前累计编号: 867
处理完成: 1-19 | 切出 147 张 -> 当前累计编号: 1014
处理完成: 1-2 | 切出 147 张 -> 当前累计编号: 1161
处理完成: 1-20 | 切出 149 张 -> 当前累计编号: 1310
处理完成: 1-21 | 切出 148 张 -> 当前累计编号: 1458
处理完成: 1-22 | 切出 148 张 -> 当前累计编号: 1606
处理完成: 1-23 | 切出 145 张 -> 当前累计编号: 1751
处理完成: 1-24 | 切出 139 张 -> 当前累计编号: 1890
处理完成: 1-25 | 切出 144 张 -> 当前累计编号: 2034
处理完成: 1-26 | 切出 145 张 -> 当前累计编号: 2179
处理完成: 1-27 | 切出 149 张 -> 当前累计编号: 2328
处理完成: 1-29 | 切出 147 张 -> 当前累计编号: 2475
处理完成: 1-3 | 切出 144 张 -> 当前累计编号: 2619
处理完成: 1-5 | 切出 146 张 -> 当前累计编号: 2765
处理完成: 1-6 | 切出 146 张 -> 当前累计编号: 2911
处理完成: 1-7 | 切出 149 张 -> 当前累计编号: 3060
处理完成: 1-8 | 切出 144 张 -> 当前累计编号: 3204
处理完成: 1-9 | 切出 141 张 -> 当前累计编号: 3345

--- 正在处理类别: 2 ---
处理完成: 2-1 | 切出 146 张 -> 当前累计编号: 146
处理完成: 2-